### LangChain Basics - Complex / Multiple Chains (LCEL Format)

In [ ]:

import os

from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import LLMChain, SequentialChain


In [ ]:
load_dotenv()

azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_deployment = os.getenv("AZURE_DEPLOYMENT_NAME")
azure_api_version = os.getenv("AZURE_API_VERSION")

In [ ]:
llm = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    azure_deployment=azure_deployment,
    api_version=azure_api_version,
    temperature=0.0,
    max_tokens=1000
)

In [ ]:
template1 = """
    You're a helpful assistant, who has good and proficient english writing skills.
    Write a Blog Outline for the following topic:
    Topic: {topic}
"""

template2 = """
    You're a helpful assistant, who has good and proficient english writing skills.
    Write a Blog Post based on the following outline:
    Outline: {outline}
"""

template3 = """
    You're a helpful assistant, who has good and proficient language skills including Translating.
    
    Translate the following text to Marathi language:
    {blog_post}
"""

prompt1 = PromptTemplate(
    input_variables=["topic"],
    template=template1
)

prompt2 = PromptTemplate(
    input_variables=["outline"],
    template=template2
)

prompt3 = PromptTemplate(
    input_variables=["blog_post"],
    template=template3
)

In [ ]:
# # Create LCEL chains using the pipe operator
# chain1 = prompt1 | llm
# chain2 = prompt2 | llm
# chain3 = prompt3 | llm

# # Combine chains using LCEL pipe operators
# sequential_chain = (
#     {"topic": RunnablePassthrough()}
#     | {"outline": chain1}
#     | {"outline": lambda x: x["outline"].content, "blog_post": chain2}
#     | {"outline": lambda x: x["outline"], "blog_post": lambda x: x["blog_post"].content, "translated_text": chain3}
#     | {"outline": lambda x: x["outline"], "blog_post": lambda x: x["blog_post"], "translated_text": lambda x: x["translated_text"].content}
# )

chain1 = LLMChain(
    llm=llm,
    prompt=prompt1,
    output_key="outline"
)

chain2 = LLMChain(
    llm=llm,
    prompt=prompt2,
    output_key="blog_post"
)

chain3 = LLMChain(
    llm=llm,
    prompt=prompt3,
    output_key="translated_text"
)

sequential_chain = SequentialChain(
    chains=[chain1, chain2, chain3],
    input_variables=["topic"],
    output_variables=["outline", "blog_post", "translated_text"],
    verbose=True
)

In [ ]:
response = sequential_chain.invoke({"topic": "Zensar AI For Leaders Program"})

In [ ]:
print("Outline:")
print(response["outline"])

In [ ]:
print("\nBlog Post:")
print(response["blog_post"])

In [ ]:
print("\nTranslated Text:")
print(response["translated_text"])